### Data Transformation

In [2]:
import os
%pwd

'c:\\Users\\nico_\\Desktop\\MLOPS\\TextSummarizer\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\nico_\\Desktop\\MLOPS\\TextSummarizer'

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class DataTransformationConfig:
    root_dir: Path
    data_path: Path
    tokenizer_name: Path


In [6]:
from src.textSummarizer.constants import *
from src.textSummarizer.utils.common import read_yaml,create_directories

In [7]:
class ConfigurationManager:
    def __init__(self,
                 config_path=CONFIG_FILE_PATH,
                 params_filepath=PARAMS_FILE_PATH):
        self.config=read_yaml(config_path)
        self.paramss=read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_data_transformation_config(self)-> DataTransformationConfig:
        config=self.config.data_transformation

        create_directories([config.root_dir])

        data_transformation_config=DataTransformationConfig(
            root_dir=config.root_dir,
            data_path=config.data_path,
            tokenizer_name=config.tokenizer_name
        )

        return data_transformation_config



In [8]:
import os
from src.textSummarizer.logging import logger
from transformers import AutoTokenizer
from datasets import load_from_disk

c:\Users\nico_\Desktop\MLOPS\TextSummarizer\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Data Transformation Component


In [9]:
class DataTransformation:
    def __init__(self, config: DataTransformationConfig):
        self.config = config
        self.tokenizer = AutoTokenizer.from_pretrained(config.tokenizer_name)

    def convert_examples_to_features(self, example_batch):
        dialogues = [str(d) for d in example_batch["dialogue"]]
        summaries = [str(s) for s in example_batch["summary"]]

        # Inputs
        input_encodings = self.tokenizer(
            dialogues,
            max_length=512,  # attention, 1024 peut faire crasher sur GPU faible
            padding="max_length",
            truncation=True
        )

        # Targets
        target_encodings = self.tokenizer(
            summaries,
            max_length=128,
            padding="max_length",
            truncation=True
        )

        # Replace [PAD] by -100 in the labels
        labels = [
            [(token if token != self.tokenizer.pad_token_id else -100) for token in label]
            for label in target_encodings["input_ids"]
        ]

        return {
            "input_ids": input_encodings["input_ids"],
            "attention_mask": input_encodings["attention_mask"],
            "labels": labels
        }

    def convert(self):
        dataset_samsum = load_from_disk(self.config.data_path)
        dataset_samsum_pt = dataset_samsum.map(self.convert_examples_to_features, batched=True)
        dataset_samsum_pt.save_to_disk(os.path.join(self.config.root_dir, "samsum_dataset"))



    

In [11]:
config=ConfigurationManager()
data_transformation_config=config.get_data_transformation_config()
data_transformation=DataTransformation(config=data_transformation_config)
data_transformation.convert()


[2025-07-23 13:22:52,175: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-07-23 13:22:52,183: INFO: common: yaml file: params.yaml loaded successfully]
[2025-07-23 13:22:52,186: INFO: common: created directory at: artifacts]
[2025-07-23 13:22:52,190: INFO: common: created directory at: artifacts/data_transformation]


Saving the dataset (1/1 shards): 100%|██████████| 819/819 [00:00<00:00, 99269.88 examples/s] 
